In [58]:
from sklearn.preprocessing import normalize;from sklearn.preprocessing import MinMaxScaler;import sklearn;from sklearn.impute import SimpleImputer;import statsmodels;import statsmodels.api as sm;from statsmodels.tsa.api import VAR
import numpy as np;import matplotlib.ticker as mticker
import scipy as sc;from scipy import linalg;import matplotlib.pyplot as plt;import numpy as np
from numpy import genfromtxt;from scipy.stats import beta
from scipy.stats import norm;from scipy.stats import gamma
from scipy.stats import invgamma;import matplotlib.pyplot as plt
import seaborn as sns;import math;import pandas as pd;import scipy as sc;from sklearn.mixture import GaussianMixture;from sklearn.mixture import BayesianGaussianMixture;import time;
import numpy as np;from sklearn.mixture import GaussianMixture;from sklearn import mixture
m= np.matmul;resh = np.reshape;norm= np.random.standard_normal;inv  = np.linalg.inv ;det = np.linalg.det
import time;import statsmodels.api as sm;from statsmodels.tsa.api import VAR
arr = np.array;zeros = np.zeros
import matplotlib.ticker as mtick
import scipy.io as sio 

In [59]:
import torch
from pathlib import Path
#from google_drive_downloader import GoogleDriveDownloader as gdd
#from tqdm import tqdm, tqdm_notebook
import torch.optim as optim


In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [60]:
DATA_PATH = '/Users/ajay/Desktop/text.csv'
#if not Path(DATA_PATH).is_file():
#    gdd.download_file_from_google_drive(
#        file_id='1zfM5E6HvKIe7f3rEt1V2gBpw5QOSSKQz',
#        dest_path=DATA_PATH,
#    )

In [61]:
from torch.utils.data import DataLoader, Dataset
from sklearn.feature_extraction.text import CountVectorizer
import torch.nn as nn
import torch.nn.functional as F


In [63]:
df

,Unnamed: 0,review,label
0,0,Once again Mr. Costner has dragged out a movie...,0
1,1,This is an example of why the majority of acti...,0
2,2,"First of all I hate those moronic rappers, who...",0
3,3,Not even the Beatles could write songs everyon...,0
4,4,Brass pictures (movies is not a fitting word f...,0
...,...,...,...
62150,62150,"I am a student of film, and have been for seve...",0
62151,62151,"Unimaginably stupid, redundant and humiliating...",0
62152,62152,"Guy is a loser. Can't get girls, needs to buil...",0
62153,62153,This 30 minute documentary Buñuel made in the ...,0


In [64]:
df  = pd.read_csv(DATA_PATH)
df.sample(5)


,Unnamed: 0,review,label
12603,12603,"An enjoyable movie, without a doubt, and very ...",1
21413,21413,Just finished with Zombie Bloodbath part one o...,1
18534,18534,"The movie is incredible, it has a sound track ...",1
12208,12208,As long as you keep in mind that the productio...,0
54254,54254,i went into this hoping it would be the \thoug...,0


In [26]:
class Sequences():
    def __init__(self, path):
        df = pd.read_csv(path)
        self.vectorizer = CountVectorizer(stop_words='english', max_df=0.99, min_df=0.005)
        self.sequences = self.vectorizer.fit_transform(df.iloc[:,1].tolist())
        self.labels = df.iloc[:,-1].tolist()
        self.token2idx = self.vectorizer.vocabulary_
        self.idx2token = {idx: token for token, idx in self.token2idx.items()}
    #### used for indexing ####    
    def __getitem__(self, i):
        return self.sequences[i, :].toarray(), self.labels[i]    
    def __len__(self):
        return self.sequences.shape[0]

In [30]:
dataset = Sequences(DATA_PATH)

In [ ]:
#df = pd.read_csv(DATA_PATH)

In [31]:
#df.iloc[:,0]

In [32]:
class BagOfWordsClassifier(nn.Module):
    def __init__(self, vocab_size, hidden1, hidden2):
        super(BagOfWordsClassifier, self).__init__()
        self.fc1 = nn.Linear(vocab_size, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, 1)
    
    def forward(self, inputs):
        x = F.relu(self.fc1(inputs.squeeze(1).float()))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

In [48]:
model = BagOfWordsClassifier(len(dataset.token2idx), 128, 64)
train_loader = DataLoader(dataset, batch_size=4096)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=0.01)

In [50]:
model.train()
n_epochs =3
b_s = int(df.shape[0]/5)
losses = []
#train_losses
total = 0 
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, batch_size=4096, shuffle=True)
    for i, batch in enumerate(dataloader):
        model.zero_grad()
        output = model(batch[0])  # Replace inputs with batch[0]
        loss = criterion(output.squeeze(), batch[1].float())        
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 3)
        optimizer.step()
        losses.append(loss.item())
        total += 1

    epoch_loss = sum(losses) #/ total
    print(epoch,epoch_loss)
    #train_losses.append(epoch_loss)
        
    

0 7.11336562037468
1 11.252426728606224
2 14.699684843420982


In [65]:
def predict_sentiment(text):
    model.eval()
    with torch.no_grad():
        test_vector = torch.LongTensor(dataset.vectorizer.transform([text]).toarray())

        output = model(test_vector)
        prediction = torch.sigmoid(output).item()

        if prediction > 0.5:
            print(f'{prediction:0.3}: Positive sentiment')
        else:
            print(f'{prediction:0.3}: Negative sentiment')

In [66]:
test_text = """
This poor excuse for a movie is terrible. It has been 'so good it's bad' for a
while, and the high ratings are a good form of sarcasm, I have to admit. But
now it has to stop. Technically inept, spoon-feeding mundane messages with the
artistic weight of an eighties' commercial, hypocritical to say the least, it
deserves to fall into oblivion. Mr. Derek, I hope you realize you are like that
weird friend that everybody know is lame, but out of kindness and Christian
duty is treated like he's cool or something. That works if you are a good
decent human being, not if you are a horrible arrogant bully like you are. Yes,
Mr. 'Daddy' Derek will end on the history books of the internet for being a
delusional sour old man who thinks to be a good example for kids, but actually
has a poster of Kim Jong-Un in his closet. Destroy this movie if you all have a
conscience, as I hope IHE and all other youtube channel force-closed by Derek
out of SPITE would destroy him in the courts.This poor excuse for a movie is
terrible. It has been 'so good it's bad' for a while, and the high ratings are
a good form of sarcasm, I have to admit. But now it has to stop. Technically
inept, spoon-feeding mundane messages with the artistic weight of an eighties'
commercial, hypocritical to say the least, it deserves to fall into oblivion.
Mr. Derek, I hope you realize you are like that weird friend that everybody
know is lame, but out of kindness and Christian duty is treated like he's cool
or something. That works if you are a good decent human being, not if you are a
horrible arrogant bully like you are. Yes, Mr. 'Daddy' Derek will end on the
history books of the internet for being a delusional sour old man who thinks to
be a good example for kids, but actually has a poster of Kim Jong-Un in his
closet. Destroy this movie if you all have a conscience, as I hope IHE and all
other youtube channel force-closed by Derek out of SPITE would destroy him in
the courts.
"""
predict_sentiment(test_text)

3.46e-06: Negative sentiment


In [67]:
test_text = '''
This Movie sucks!

'''
predict_sentiment(test_text)

0.107: Negative sentiment


In [68]:
test_text = '''
This Movie is so good

'''
predict_sentiment(test_text)

0.567: Positive sentiment


In [55]:
test_text = '''
This Movie is so moving, it made me feel elated and gave me a sense of purpose in life. 
'''
predict_sentiment(test_text)

0.747: Positive sentiment


In [57]:
test_text = '''
This Movie had a truly horrible impact on my health, it made me feel depressed and had me lose my sense of purpose in life 
'''
predict_sentiment(test_text)

0.493: Negative sentiment


In [ ]:
 
X_train   =  Y[:-1]
Y_train = Y[1:]

In [33]:
progress_bar = tqdm_notebook(train_loader, leave=False)
losses = []
total = 0
for inputs, target in progress_bar:
    model.zero_grad()
    output = model(inputs)
    loss = criterion(output.squeeze(), target.float())
    loss.backward()


/home/aj/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/16 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
for inputs, target in progress_bar:
    print(inputs,target)


In [ ]:
type(progress_bar)